# Inputs

In [1]:
splitPartCount = 5
splitSeed = 27
hoursPerWindow = 1

`functionize-notebook` has modified this notebook during execution. The following variables have been injected:

- hoursPerWindow: 6


# Preproccess

## read data

In [3]:
from utils.class_patient import Patients


patients = Patients.loadPatients()
len(patients)

1213

## remove missing

In [4]:
# fill measures whose null represent false value

from constants import NULLABLE_MEASURES


nullableMeasures = NULLABLE_MEASURES

for measureName in nullableMeasures:
    patients.fillMissingMeasureValue(measureName, 0)

In [5]:
# remove measures with less than 80% of data

measures = patients.getMeasures()

for measure, count in measures.items():
    if count < len(patients) * 80 / 100:
        patients.removeMeasures([measure])
        print(measure, count)

pco2 917
ph 954
po2 917
albumin 406
hba1c 326
lymphocyte 446
height 415
urine-ketone 294
crp 19


In [6]:
# remove patients with less than 80% of data

patients.removePatientByMissingFeatures()
len(patients)

1209

## split patients

In [7]:
splitedPatients = patients.split(splitPartCount, splitSeed)

len(splitedPatients[0])

242

In [8]:
from pandas import Timedelta


splitedPatients[0].getMeasuresBetween(Timedelta(-6), Timedelta(1))

,subject_id,hadm_id,stay_id,akd,ag,age,bg,bicarbonate,bun,calcium,...,race,rr,saps2,sbp,scr,sofa,use_NaHCO3,uti,wbc,weight
0,19054290,20046699,30643955,False,NaN,65,NaN,NaN,NaN,NaN,...,WHITE,NaN,34,NaN,NaN,3,0.0,0,NaN,74.50
0,14866589,20066152,33060916,False,NaN,37,NaN,NaN,NaN,NaN,...,WHITE,NaN,22,NaN,NaN,2,0.0,0,NaN,NaN
0,14849360,20152551,32817197,False,NaN,59,NaN,NaN,NaN,NaN,...,UNABLE TO OBTAIN,NaN,31,NaN,NaN,7,0.0,0,NaN,NaN
0,16171124,20167052,38239449,False,NaN,37,NaN,NaN,NaN,NaN,...,HISPANIC/LATINO - DOMINICAN,NaN,15,NaN,NaN,2,0.0,0,NaN,NaN
0,16815664,20240670,37751533,False,NaN,42,NaN,NaN,NaN,NaN,...,BLACK/AFRICAN AMERICAN,NaN,27,NaN,NaN,2,NaN,0,NaN,43.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,15517352,29801212,32814591,False,NaN,20,NaN,NaN,NaN,NaN,...,WHITE,NaN,30,NaN,NaN,4,NaN,0,NaN,NaN
0,10383045,29899941,39755932,False,NaN,58,NaN,NaN,NaN,NaN,...,WHITE,NaN,23,NaN,NaN,1,0.0,0,NaN,59.40
0,15679298,29905973,39834726,False,NaN,50,NaN,NaN,NaN,NaN,...,WHITE,NaN,32,NaN,NaN,3,0.0,0,NaN,47.40
0,10912213,29911812,34040470,False,NaN,47,NaN,NaN,NaN,NaN,...,WHITE,NaN,23,NaN,NaN,2,0.0,0,NaN,89.05


In [9]:
splitedPatients = patients.split(splitPartCount, splitSeed)


def trainTest():
    for i in range(splitedPatients.__len__()):
        testPatients = splitedPatients[i]

        trainPatientsList = splitedPatients[:i] + splitedPatients[i + 1 :]
        trainPatients = Patients(patients=[])
        for trainPatientsElem in trainPatientsList:
            trainPatients += trainPatientsElem

        yield trainPatients, testPatients


def trainValTest():
    for i in range(splitedPatients.__len__()):
        testPatients = splitedPatients[i]

        trainPatientsList = splitedPatients[:i] + splitedPatients[i + 1 :]
        trainPatients = Patients(patients=[])
        for trainPatientsElem in trainPatientsList:
            trainPatients += trainPatientsElem

        *trainPatients, valPatients = trainPatients.split(5, 27)
        tmpPatients = Patients(patients=[])
        for trainPatientsElem in trainPatients:
            tmpPatients += trainPatientsElem
        trainPatients = tmpPatients

        yield trainPatients, valPatients, testPatients

In [10]:
for trainPatients, testPatients in trainTest():
    print(len(trainPatients.patientList), len(testPatients.patientList))

967 242
967 242
967 242
967 242
968 241


In [11]:
from constants import CATEGORICAL_MEASURES, TEMP_PATH
from utils.prepare_data import patientsToNumpy


dataset = []
for i, (trainPatients, testPatients) in enumerate(trainTest()):
    # trainPatients.fillMissingMeasureValue(CATEGORICAL_MEASURES, 0)
    npTrainX, categoryEncoder, numericEncoder, oulier, columns = patientsToNumpy(
        trainPatients, hoursPerWindow, CATEGORICAL_MEASURES
    )

    # testPatients.fillMissingMeasureValue(CATEGORICAL_MEASURES, 0)
    npTestX, *_ = patientsToNumpy(
        testPatients,
        hoursPerWindow,
        CATEGORICAL_MEASURES,
        columns,
        categoryEncoder,
        numericEncoder,
        oulier,
    )
    trainY = [p.akdPositive for p in trainPatients]
    testY = [p.akdPositive for p in testPatients]

    dataset.append((npTrainX, trainY, npTestX, testY))

    print(f"Dataset {i} created")
    pass

Dataset 0 created


Dataset 1 created


Dataset 2 created


Dataset 3 created


Dataset 4 created


In [12]:
import pickle


(TEMP_PATH / f"train-test-set-{splitPartCount}-{splitSeed}-{hoursPerWindow}.pkl").write_bytes(
    pickle.dumps(dataset)
)

6912494

In [13]:
from constants import CATEGORICAL_MEASURES, TEMP_PATH
from utils.prepare_data import patientsToNumpy


datavalset = []
for i, (trainPatients, valPatients, testPatients) in enumerate(trainValTest()):
    # trainPatients.fillMissingMeasureValue(CATEGORICAL_MEASURES, 0)
    npTrainX, categoryEncoder, numericEncoder, oulier, columns = patientsToNumpy(
        trainPatients, hoursPerWindow, CATEGORICAL_MEASURES
    )

    # testPatients.fillMissingMeasureValue(CATEGORICAL_MEASURES, 0)
    npTestX, *_ = patientsToNumpy(
        testPatients,
        hoursPerWindow,
        CATEGORICAL_MEASURES,
        columns,
        categoryEncoder,
        numericEncoder,
        oulier,
    )

    npValX, *_ = patientsToNumpy(
        valPatients,
        hoursPerWindow,
        CATEGORICAL_MEASURES,
        columns,
        categoryEncoder,
        numericEncoder,
        oulier,
    )

    trainY = [p.akdPositive for p in trainPatients]
    testY = [p.akdPositive for p in testPatients]
    valY = [p.akdPositive for p in valPatients]

    datavalset.append((npTrainX, trainY, npValX, valY, npTestX, testY))

    print(f"Dataset {i} created")
    pass

Dataset 0 created


Dataset 1 created


Dataset 2 created


Dataset 3 created


Dataset 4 created


In [14]:
import pickle

(TEMP_PATH / f"train-val-test-set-{splitPartCount}-{splitSeed}-{hoursPerWindow}.pkl").write_bytes(
    pickle.dumps(datavalset)
)

6816024

# LSTM

In [15]:
import pickle
from constants import TEMP_PATH

# load if not exist dataset
if "dataset" not in locals():
    dataset = pickle.loads(
        (TEMP_PATH / f"dataset-{splitPartCount}-{splitSeed}-{hoursPerWindow}.pkl").read_bytes()
    )
   

In [16]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

def createModel(timeSteps, features):
    model = Sequential()
    model.add(LSTM(64, input_shape=(timeSteps, features), return_sequences=True))
    model.add(Dropout(0.5))
    model.add(LSTM(32))
    model.add(Dropout(0.5))
    model.add(Dense(16, activation="relu"))
    model.add(Dense(1, activation="sigmoid"))

    optimizer = Adam(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=["AUC", "accuracy", "precision", "recall"])
    return model

2024-06-07 12:35:57.713929: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-07 12:35:57.732009: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


2024-06-07 12:35:58.009046: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [17]:
# crossfold
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping


loses = []
aucs = []
accuracies = []
precisions = []
recals = []
for i, (npTrainX, trainY, npTestX, testY) in enumerate(dataset):
    npTrainX = np.nan_to_num(npTrainX, nan=0)
    npTestX = np.nan_to_num(npTestX, nan=0)

    model = createModel(npTrainX.shape[1], npTrainX.shape[2])

    neg, pos = np.bincount(trainY)
    weight0 = (1 / neg) * (len(trainY)) / 2.0
    weight1 = (1 / pos) * (len(trainY)) / 2.0
    weight = {0: weight0, 1: weight1}

    early_stopping = EarlyStopping(
        monitor="val_loss", patience=10, restore_best_weights=True
    )

    trainY = np.array(trainY)
    model.fit(
        npTrainX,
        trainY,
        epochs=50,
        batch_size=32,
        validation_split=0.2,
        class_weight=weight,
        callbacks=[early_stopping],
    )

    testY = np.array(testY)
    loss, auc, accuracy, precison, recal = model.evaluate(npTestX, testY)
    loses.append(loss)
    aucs.append(auc)
    accuracies.append(accuracy)
    precisions.append(precison)
    recals.append(recal)

    pass

print("Loses:", loses, np.mean(loses), np.std(loses))
print("AUCs:", aucs, np.mean(aucs), np.std(aucs))
print("Accuracies:", accuracies, np.mean(accuracies), np.std(accuracies))
print("Precisions:", precisions, np.mean(precisions), np.std(precisions))
print("Recals:", recals, np.mean(recals), np.std(recals))

Epoch 1/50


2024-06-07 12:35:58.364623: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-07 12:35:58.379365: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
/home/tu/codepy/hust.year2023.PredictingRiskDiabeticKetoacidosis-associatedKidneyInjury/.venv/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer usi

 1/25 ━━━━━━━━━━━━━━━━━━━━ 28s 1s/step - AUC: 0.4375 - accuracy: 0.5000 - loss: 0.6975 - precision: 0.3333 - recall: 0.3333

25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - AUC: 0.6545 - accuracy: 0.6241 - loss: 0.6680 - precision: 0.5193 - recall: 0.4867 - val_AUC: 0.7673 - val_accuracy: 0.7062 - val_loss: 0.5950 - val_precision: 0.6000 - val_recall: 0.6575


Epoch 2/50


...

 1/25 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - AUC: 0.9434 - accuracy: 0.8750 - loss: 0.3281 - precision: 0.8750 - recall: 0.8750

25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - AUC: 0.9047 - accuracy: 0.8489 - loss: 0.3925 - precision: 0.7995 - recall: 0.8418 - val_AUC: 0.7531 - val_accuracy: 0.7216 - val_loss: 0.6561 - val_precision: 0.6400 - val_recall: 0.6400


1/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - AUC: 0.7871 - accuracy: 0.8125 - loss: 0.5536 - precision: 0.7778 - recall: 0.8750

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 900us/step - AUC: 0.7849 - accuracy: 0.7607 - loss: 0.5680 - precision: 0.6799 - recall: 0.7731


Loses: [0.5774437189102173, 0.5508691668510437, 0.5418391823768616, 0.5783491730690002, 0.5853409171104431] 0.5667684316635132 0.01713047948278086
AUCs: [0.7607591152191162, 0.805871844291687, 0.8243465423583984, 0.7644834518432617, 0.7765320539474487] 0.7863986015319824 0.024715417059188442
Accuracies: [0.7066115736961365, 0.7355371713638306, 0.7727272510528564, 0.6900826692581177, 0.7385892271995544] 0.7287095785140991 0.028503118824449787
Precisions: [0.6132075190544128, 0.642201840877533, 0.6754385828971863, 0.5945945978164673, 0.6481481194496155] 0.6347181320190429 0.028167818685168485
Recals: [0.6842105388641357, 0.7368420958518982, 0.8105263113975525, 0.6875, 0.7368420958518982] 0.731184208393097 0.045769047143479256


## Static validate 

In [18]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam


def createModel(timeSteps, features):
    model = Sequential()
    model.add(LSTM(64, input_shape=(timeSteps, features), return_sequences=True))
    model.add(Dropout(0.5))
    model.add(LSTM(32))
    model.add(Dropout(0.5))
    model.add(Dense(16, activation="relu"))
    model.add(Dense(1, activation="sigmoid"))

    optimizer = Adam(learning_rate=0.001)
    model.compile(
        optimizer=optimizer,
        loss="binary_crossentropy",
        metrics=["AUC", "accuracy", "precision", "recall"],
    )
    return model

In [19]:
# crossfold
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping

if "datavalset" not in locals():
    datavalset = pickle.loads(
        (TEMP_PATH / f"train-val-test-set-{splitPartCount}-{splitSeed}-{hoursPerWindow}.pkl").read_bytes()
    )

loses = []
aucs = []
accuracies = []
precisions = []
recals = []
for i, (npTrainX, trainY, npValX, valY, npTestX, testY) in enumerate(datavalset):
    npTrainX = np.nan_to_num(npTrainX, nan=0)
    npValX = np.nan_to_num(npValX, nan=0)
    npTestX = np.nan_to_num(npTestX, nan=0)

    model = createModel(npTrainX.shape[1], npTrainX.shape[2])

    neg, pos = np.bincount(trainY)
    weight0 = (1 / neg) * (len(trainY)) / 2.0
    weight1 = (1 / pos) * (len(trainY)) / 2.0
    weight = {0: weight0, 1: weight1}

    early_stopping = EarlyStopping(
        monitor="val_loss", patience=10, restore_best_weights=True
    )

    trainY = np.array(trainY)
    valY = np.array(valY)
    testY = np.array(testY)

    model.fit(
        npTrainX,
        trainY,
        epochs=50,
        batch_size=32,
        validation_data=(npValX, valY),
        class_weight=weight,
        callbacks=[early_stopping],
    )

    loss, auc, accuracy, precison, recal = model.evaluate(npTestX, testY)
    loses.append(loss)
    aucs.append(auc)
    accuracies.append(accuracy)
    precisions.append(precison)
    recals.append(recal)

    pass

print("Loses:", loses, np.mean(loses), np.std(loses))
print("AUCs:", aucs, np.mean(aucs), np.std(aucs))
print("Accuracies:", accuracies, np.mean(accuracies), np.std(accuracies))
print("Precisions:", precisions, np.mean(precisions), np.std(precisions))
print("Recals:", recals, np.mean(recals), np.std(recals))

Epoch 1/50


 1/25 ━━━━━━━━━━━━━━━━━━━━ 27s 1s/step - AUC: 0.4437 - accuracy: 0.4688 - loss: 0.6850 - precision: 0.2857 - recall: 0.3636

25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - AUC: 0.6254 - accuracy: 0.6007 - loss: 0.6738 - precision: 0.4862 - recall: 0.5734 - val_AUC: 0.7304 - val_accuracy: 0.6425 - val_loss: 0.6554 - val_precision: 0.5315 - val_recall: 0.7763


Epoch 2/50


 1/25 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - AUC: 0.8229 - accuracy: 0.8125 - loss: 0.5873 - precision: 0.7500 - recall: 0.7500

...

 1/25 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - AUC: 0.8750 - accuracy: 0.8125 - loss: 0.4319 - precision: 0.6429 - recall: 0.9000

25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - AUC: 0.9111 - accuracy: 0.8362 - loss: 0.3766 - precision: 0.7415 - recall: 0.8638 - val_AUC: 0.7872 - val_accuracy: 0.7358 - val_loss: 0.6016 - val_precision: 0.6471 - val_recall: 0.7237


1/8 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - AUC: 0.7578 - accuracy: 0.7500 - loss: 0.6008 - precision: 0.7857 - recall: 0.6875

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 872us/step - AUC: 0.7748 - accuracy: 0.7256 - loss: 0.5662 - precision: 0.6853 - recall: 0.6008


Loses: [0.5938394665718079, 0.5592325329780579, 0.4971330165863037, 0.5666549205780029, 0.5807310342788696] 0.5595181941986084 0.0333792126121997
AUCs: [0.747941255569458, 0.7998567819595337, 0.8380236029624939, 0.777254581451416, 0.7634463310241699] 0.7853045105934143 0.031399571657557517
Accuracies: [0.6983470916748047, 0.7272727489471436, 0.7644628286361694, 0.7148760557174683, 0.7053942084312439] 0.7220705866813659 0.023310219873918153
Precisions: [0.6037735939025879, 0.6198347210884094, 0.6899999976158142, 0.6391752362251282, 0.6395348906517029] 0.6384636878967285 0.0290106395017249
Recals: [0.6736842393875122, 0.7894737124443054, 0.7263157963752747, 0.6458333134651184, 0.5789473652839661] 0.6828508853912354 0.071425339000363
